<a href="https://colab.research.google.com/github/CarolineLaure/One_Qubit_TensorFlow_example/blob/master/three_level_system_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now let have the Hamiltonian of the form H = - $\Delta_1$|e><e| - $\delta_{total}$|r><r| - $\Omega_1/2$(|g><e| + |e><g|)- $\Omega_2/2$(|e><r| + |r><e|) where |g>, |e>, |r> are respectively the ground, intermediate and the Rydberg state. The goal is to optimize the overlap between the ground and the Rydberg state

In [2]:
pip install qutip

     |████████████████████████████████| 4.9MB 2.7MB/s 
  Created wheel for qutip: filename=qutip-4.5.1-cp36-cp36m-linux_x86_64.whl size=14080966 sha256=16563532e21711844dda64a773929bbb1c5ef729e72067a8042edb969176012b
  Stored in directory: /root/.cache/pip/wheels/8e/50/3d/29fb07fa1c0fbca787cff4376446b88a58bf628e324cee6c95
Successfully built qutip


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
import qutip as qt
import matplotlib as plt 
print(tf.__version__) 

2.2.0


In [0]:
start = time.time()

# De leseleuc parameters

delta_1 = 2 * np.pi * 560 * 10**6
sigma_total = 2 * np.pi * 13 * 10**6
Omega_1 = 2 * np.pi * 60 * 10**6
Omega_2 = 2 * np.pi * 36 * 10**6

In [23]:
class Propagator:
    def __init__(self, no_of_steps, dim, delta_t, delta_1, sigma_total, Omega_1, Omega_2):

        # Define my Hamiltonian

        self.delta_1 = delta_1
        self.sigma_total = sigma_total
        self.Omega_1 = Omega_1
        self.Omega_2 = Omega_2
        self.delta_t = delta_t
        
        g, e, r = qt.qutrit_basis()

        Hamiltonian = - self.delta_1 * e * e.dag() - self.sigma_total * r * r.dag() - self.Omega_1/2 * (g * e.dag() + e * g.dag()) 
        - self.Omega_2/2 * (e * r.dag() + r * e.dag())


        self.delta_t=delta_t
        self.dim=3
        self.initial_state= tf.constant(g, dtype=tf.complex128)
        self.final_state= tf.constant(r, dtype=tf.complex128)
        

        self.ctrl_amplitude = tf.Variable(tf.zeros([no_of_steps, 4], dtype=tf.float64), dtype=tf.float64)  # control amplitude $\Omega$

        self.generators =  tf.stack([
                                     tf.constant(- delta_1 * e * e.dag(), dtype=tf.complex128),
                                     tf.constant(- sigma_total * r * r.dag(), dtype=tf.complex128),
                                     tf.constant(- Omega_1 * 0.5 * (g * e.dag() + e * g.dag()), dtype=tf.complex128),
                                     tf.constant(- Omega_2 * 0.5 * (e * r.dag() + r * e.dag()), dtype=tf.complex128),
                          
                                     ])

        self.contraction_array=[]
        contraction_array_length = int(np.floor(np.log2(no_of_steps)))
        temp_no_of_steps= no_of_steps

        for i in range(contraction_array_length):
          self.contraction_array.append(bool(np.mod(temp_no_of_steps, 2))) 
          temp_no_of_steps = np.floor(temp_no_of_steps/2)

    def exponential(self):
          regularize_amplitudes = 1/np.sqrt(2)*tf.math.tanh(self.ctrl_amplitude)
          exponent = -1j*self.delta_t*(tf.linalg.tensordot(tf.cast(regularize_amplitudes, dtype=tf.complex128), self.generators, 1))
          return tf.linalg.expm(exponent)

    def propagate(self):
          step_exps=self.exponential()
          for is_odd in self.contraction_array:
            if is_odd:
              odd_exp=step_exps[-1, :, :]
              step_exps = tf.linalg.matmul(step_exps[1::2, :, :], step_exps[0:-1:2, :, :])
              step_exps = tf.concat([step_exps[0:-1, :, :], [tf.linalg.matmul(odd_exp, step_exps[-1, :, :])]], 0)
            else:
              step_exps = tf.linalg.matmul(step_exps[1::2, :, :], step_exps[0::2, :, :])
          return tf.squeeze(step_exps)

    @tf.function
    def infidelity(self):
          propagator=self.propagate()
          intermediate_state = tf.linalg.matvec(propagator, tf.transpose(self.initial_state))
          overlap = tf.tensordot(tf.math.conj(intermediate_state), self.final_state, axes=1)
          return 1-tf.math.real(tf.math.conj(overlap)*overlap)

propagator = Propagator(2000, 3, 10**-9, delta_1, sigma_total, Omega_1, Omega_2)

optimizer = tf.keras.optimizers.Adam(0.01)

propagator.ctrl_amplitude.assign(
    tf.random.uniform([2000, 4], -1, 1, dtype=tf.float64)
)

#propagator.infidelity()

@tf.function  
def optimization_step():
  with tf.GradientTape() as tape:
    infidelity = propagator.infidelity()
  gradients = tape.gradient(infidelity, [propagator.ctrl_amplitude])
  optimizer.apply_gradients(zip(gradients, [propagator.ctrl_amplitude]))
  return infidelity

overlap1 = []
steps = range(100)
for step in steps:
  current_infidelity = optimization_step()
  #print('step %2d: infidelity=%2.5f' % (step, current_infidelity))
  overlap1.append(current_infidelity[0][0])
tf.print(overlap1) 

#final_pulse = propagator.ctrl_amplitude  
       
end = time.time()
print('Run Time: %2.4f seconds' %(end-start))

np.savetxt("Overlap_infidelity.txt", overlap1)
# #downloads to local machine: from google.colab import files files.download('Overlap_infidelity.txt')
from google.colab import files
files.download('Overlap_infidelity.txt')


[0.0704378899185748,
 0.25100389599552608,
 0.59853746686066756,
 0.88673330306271281,
 0.47645467514842044,
 0.91572976089305824,
 0.76748905840492054,
 0.7843665442179677,
 0.35103148966483166,
 0.42196008378557703,
 0.74857695321943929,
 0.86965949298154954,
 0.9328081673771349,
 0.98447355424328542,
 0.93456960201987549,
 0.81946494502635869,
 0.61972798138002472,
 0.877709420994359,
 0.9640567659882594,
 0.73415287797502793,
 0.77209162243913532,
 0.44626619890157504,
 0.79812276539522276,
 0.2512476288819514,
 0.82902385298514869,
 0.76068645434420512,
 0.34412746537146588,
 0.55205601335082866,
 0.50613681299818081,
 0.50027540414992722,
 0.3040337817640959,
 0.23421908402322911,
 0.38372156565465709,
 0.086323131666872222,
 0.22839819376319348,
 0.20923888971584015,
 0.11216185211212581,
 0.05043874556706518,
 0.1486761452691896,
 0.031705017822784454,
 0.10439057468592616,
 0.056446825346992968,
 0.039933094952177894,
 0.073622675214633171,
 0.018502828582300124,
 0.0572717878